## 统计类特征提取
该文本分类任务是预测案件金额类别，因此案件文本中出现的金额是重要的。于是，我使用正则表达式匹配出案件文本中出现的所有金额，然后对同一个案件中出现的所有金额进行统计，包括：求和、最小值、最大值、最大最小差值、平均值、标准差。

这里还可以做的事情有：

统计案件文本的词的数目 <br>
利用案件中的一些关键词做特征，例如：酒驾、毒品等 <br>
案件文本中出现的日期 <br>
案件文本中出现的地点 <br>

下面我们主要提取金额类的特征

In [1]:
import sys
sys.path.append("D:/ML_Study/2017-CCF-BDCI-AIJudge")
import re
import numpy as np
import pandas as pd
from config.db_config import Config
from utils import LOGGER
import codecs
config = Config()

In [3]:
def load_data(data_path):
    df_tr = []
    LOGGER.log('For train.txt:')
    for i, line in enumerate(codecs.open(data_path, encoding='utf-8')):
        if i % 1000 == 1:
            LOGGER.log('iter = %d' % i)
        segs = line.split('\t')
        row = {}
        row['id'] = segs[0]
        row['raw_content'] = segs[1].strip()
        df_tr.append(row)
    data_df = pd.DataFrame(df_tr)
    return data_df

In [4]:
train_df = load_data(config.train_data_path)
test_df = load_data(config.test_data_path)
df = pd.concat([train_df, test_df]).reset_index(drop=True)
df.head()

2019-03-26 17:09:46 For train.txt:
2019-03-26 17:09:46 iter = 1
2019-03-26 17:09:46 For train.txt:


,id,raw_content
0,16,公诉机关霍邱县人民检察院。被告人许某甲，男，1975年9月20日生。2012年12月17日因...
1,32,公诉机关海口市龙华区人民检察院。被告人王某乙。海口市龙华区人民检察院以海龙检公刑诉（2014...
2,41,公诉机关广东省潮州市人民检察院。被告人覃学彬（自报），男，1980年1月8日出生，广西壮族自...
3,57,原公诉机关榆林市榆阳区人民检察院。上诉人（原审被告人）高某某，2012年10月22日因涉嫌犯...
4,60,原公诉机关榆阳区人民检察院。上诉人（原审被告人）刘某，男，汉族，陕西省横山县，小学文化，货车...


In [37]:
amt_list = []
for i, rows in df.iterrows():
    if i % 1000 == 1:
        LOGGER.log('iter = %d' % i)
    id = rows["id"]
    raw_document = rows['raw_content']
    row_amount_list = re.findall(u'(\d*\.?\d+)[元]', raw_document)
    for amount in row_amount_list:
        amt_list.append([id,float(amount)])

2019-03-26 17:48:35 iter = 1


In [38]:
amt_df = pd.DataFrame(amt_list,columns=["id","amt"])
amt_df.head()

,id,amt
0,32,550.0
1,32,550.0
2,32,550.0
3,57,6400.0
4,57,3800.0


In [76]:
feat_amt = amt_df.groupby(by="id")['amt'].agg([sum,max,min,np.ptp,np.mean,np.std]).reset_index()
feat_amt.head()

,id,sum,max,min,ptp,mean,std
0,119,2475.0,2475.0,2475.0,0.0,2475.000000,NaN
1,121,2500.0,2500.0,2500.0,0.0,2500.000000,NaN
2,123,7000.0,7000.0,7000.0,0.0,7000.000000,NaN
3,128,2017.4,2017.4,2017.4,0.0,2017.400000,NaN
4,211,4684.4,3684.4,500.0,3184.4,1561.466667,1838.514197


In [65]:
feat_amt.columns = ['amt_{0}'.format(col) for col in feat_amt.columns]

In [66]:
feat_amt.head()

,amt_sum,amt_max,amt_min,amt_ptp,amt_mean,amt_std
0,2475.0,2475.0,2475.0,0.0,2475.000000,NaN
1,2500.0,2500.0,2500.0,0.0,2500.000000,NaN
2,7000.0,7000.0,7000.0,0.0,7000.000000,NaN
3,2017.4,2017.4,2017.4,0.0,2017.400000,NaN
4,4684.4,3684.4,500.0,3184.4,1561.466667,1838.514197


In [69]:
config.feat_amt = "D:/ML_Study/2017-CCF-BDCI-AIJudge/data/output/feature/amt/amt_21w.csv"
feat_amt.to_csv(config.feat_amt,index=None)

In [73]:
float('%.2f' %(1561.466667))

1561.47